In [ ]:
import pandas as pd
import geopandas as gpd
import matplotlib.pyplot as plt
import h3

from shapely.geometry import Point
from shapely.geometry import Polygon

import contextily as ctx

#from geopy.distance import geodesic
from datetime import datetime


In [ ]:
import itertools

In [ ]:
filename_DD = '../data/nextbike/trips_Dresden with hexagons 2025-03-17_12-01-41.pkl'
filename_FB = '../data/nextbike/trips_Freiburg with hexagons 2025-03-17_12-01-41.pkl'
df_DD = pd.read_pickle(filename_DD)
df_FB = pd.read_pickle(filename_FB)

In [ ]:
df_DD

In [ ]:
(df_DD.hex_id_rent==df_DD.hex_id_return).value_counts()

In [ ]:
(df_FB.hex_id_rent==df_FB.hex_id_return).value_counts()

In [ ]:
df_DD

In [ ]:
df_DD['datetime_30min_rent'] = df_DD.datetime_rent.dt.floor("30min")
df_FB['datetime_30min_rent'] = df_FB.datetime_rent.dt.floor("30min")
df_DD['datetime_30min_return'] = df_DD.datetime_return.dt.floor("30min")
df_FB['datetime_30min_return'] = df_FB.datetime_return.dt.floor("30min")


In [ ]:
df_tmp = df_DD.copy()

In [ ]:
all_hex_ids = set(df_tmp['hex_id_return'])

In [ ]:
all_hex_ids.update(df_tmp['hex_id_rent'])

In [ ]:
all_times = df_tmp['datetime_30min_rent'].unique()

In [ ]:
full_index = pd.DataFrame(itertools.product(all_hex_ids, all_times), columns=['hex_id', 'datetime_30min'])

In [ ]:
trip_counts_rent = df_tmp.groupby(['hex_id_rent', 'datetime_30min_rent']).size().reset_index(name='rent_count')

In [ ]:
trip_counts_return = df_tmp.groupby(['hex_id_return', 'datetime_30min_return']).size().reset_index(name='return_count')

In [ ]:
trip_counts_rent = trip_counts_rent.rename(columns={'hex_id_rent':'hex_id', 'datetime_30min_rent':'datetime_30min'})


In [ ]:
trip_counts_return = trip_counts_return.rename(columns={'hex_id_return':'hex_id', 'datetime_30min_return':'datetime_30min'})


In [ ]:
full_index = full_index.merge(trip_counts_rent, how='left', on=['hex_id', 'datetime_30min']).fillna(0)

In [ ]:
# full_index

In [ ]:
# full_index.loc[full_index.hex_id=="871f1b093ffffff", ['datetime_30min', 'rent_count']].sort_values("rent_count")

In [ ]:
# for id in full_index.hex_id.unique():
#     full_index.loc[full_index.hex_id==id, ['datetime_30min', 'rent_count']].plot(x='datetime_30min', y='rent_count')


In [ ]:
full_index = full_index.merge(trip_counts_return, how='left', on=['hex_id', 'datetime_30min']).fillna(0)

In [ ]:
full_index

In [ ]:
(full_index.rent_count==0).value_counts(normalize=True)

In [ ]:
(full_index.return_count==0).value_counts(normalize=True)

In [ ]:
grouping = full_index.groupby('hex_id').rent_count.sum().sort_values(ascending=False)

In [ ]:
borderline = 100

In [ ]:
len(grouping)

In [ ]:
len(grouping[grouping>borderline])

In [ ]:
sum(grouping)

In [ ]:
sum(grouping[grouping>borderline])

In [ ]:
sum(grouping[grouping<=borderline])

In [ ]:
sum(grouping[grouping>borderline])/sum(grouping)

In [ ]:
hoo_small_hexagons = grouping[grouping<=borderline].index

In [ ]:
hoo_small_hexagons

In [ ]:
full_index = full_index[~full_index.hex_id.isin(hoo_small_hexagons)]

In [ ]:
(full_index.rent_count==0).value_counts(normalize=True)

In [ ]:
(full_index.return_count==0).value_counts(normalize=True)

In [ ]:
full_index_DD = full_index.copy()

In [ ]:
full_index_FB = full_index.copy()

In [ ]:
full_index_FB

In [ ]:
full_index_FB.loc[full_index_FB.hex_id=="871f81c99ffffff", ['datetime_30min', 'rent_count', 'return_count']].plot(x='datetime_30min', y=['rent_count', 'return_count'])

# save

In [ ]:
time = datetime.now().strftime('%Y-%m-%d_%H-%M-%S')
full_index_DD.to_pickle(f'../data/nextbike/demand_supply_Dresden {time}.pkl')
full_index_FB.to_pickle(f'../data/nextbike/demand_supply_Freiburg {time}.pkl')